### Import all the necessary packages

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [6]:
# initialize the initial learning rate, number of epochs to train for, and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [15]:
DIRECTORY = r"C:\Users\Aniket\Desktop\Face-Mask-Detection\dataset"

In [16]:
CATEGORIES = ["with_mask", "without_mask"]

In [17]:
# grab the list of images in our dataset directory, then initialize
# the list of data(i.e, images) and class images
print("[INFO] loading images..")

[INFO] loading images..


In [18]:
data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224,224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)

C:\Users\Aniket\anaconda3\envs\deeplearning\lib\site-packages\PIL\Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [19]:
# perform one hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [20]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [21]:
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.20,
                                               stratify=labels, random_state=42)

In [22]:
# construct the training image generator for data generation
aug = ImageDataGenerator(rotation_range=20,
                        zoom_range = 0.15,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.15,
                        horizontal_flip = True)

In [19]:
# load the mobilenetV2 network, ensuring the head FC layer sets are left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                       input_tensor=Input(shape=(224,224,3)))

9412608/9406464 [==============================] - 58s 6us/step


In [20]:
# Construct the head of the model that will be placed on top of the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [21]:
# place the head FC model on the top of the base model (this will become
# actual model we will train)
model = Model(inputs = baseModel.input, outputs=headModel)

In [22]:
# loop over all layers in the baseModel and freeze them so they will not be 
# updated during the first training process
for layer in baseModel.layers:
    layer.trainable=False

In [23]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
             metrics=["accuracy"])

[INFO] compiling model...


C:\Users\Aniket\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [25]:
# train our model
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),
             steps_per_epoch = len(trainX)//BS,
             validation_data = (testX, testY),
             validation_steps = len(testX) // BS,
             epochs = EPOCHS)

Epoch 1/20
95/95 [==============================] - 213s 2s/step - loss: 0.4077 - accuracy: 0.8487 - val_loss: 0.1529 - val_accuracy: 0.9909
Epoch 2/20
95/95 [==============================] - 193s 2s/step - loss: 0.1608 - accuracy: 0.9618 - val_loss: 0.0815 - val_accuracy: 0.9909
Epoch 3/20
95/95 [==============================] - 212s 2s/step - loss: 0.1129 - accuracy: 0.9687 - val_loss: 0.0592 - val_accuracy: 0.9909
Epoch 4/20
95/95 [==============================] - 172s 2s/step - loss: 0.0841 - accuracy: 0.9792 - val_loss: 0.0475 - val_accuracy: 0.9909
Epoch 5/20
95/95 [==============================] - 161s 2s/step - loss: 0.0641 - accuracy: 0.9845 - val_loss: 0.0420 - val_accuracy: 0.9909
Epoch 6/20
95/95 [==============================] - 164s 2s/step - loss: 0.0642 - accuracy: 0.9812 - val_loss: 0.0380 - val_accuracy: 0.9909
Epoch 7/20
95/95 [==============================] - 165s 2s/step - loss: 0.0539 - accuracy: 0.9865 - val_loss: 0.0355 - val_accuracy: 0.9909
Epoch 8/20
95

In [26]:
model.save('facemask.h5')

C:\Users\Aniket\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [2]:
from tensorflow.keras.models import load_model

In [3]:
model = load_model('facemask.h5')

In [24]:
predicts = model.predict(testX, batch_size=BS)

In [25]:
predicts = np.argmax(predicts, axis=1)

In [28]:
model.save("facemaskdetect.h5")

C:\Users\Aniket\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
